### Importing the libraries

In [1]:
from typing import Callable, Dict, List, Optional, Tuple, Type, Union, Any

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch as th

from stable_baselines3 import PPO
from stable_baselines3 import DQN
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import VecTransposeImage
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback

In [2]:
#!cd github & git clone https://github.com/mwydmuch/ViZDoom.git

### Importing  OpenAI and Doom Dependencies

In [3]:
from vizdoom import *
import random
import time
import gym
from gym import Env
from gym.spaces import Discrete, Box
import cv2

## Game Setup

### VizDoom

In [4]:
#Creating a game instance
game = DoomGame()
#Load our desired Doom configurations 
game.load_config('github/ViZDoom/scenarios/basic.cfg')
#starting up the game
#game.init()

True

## Wrapping and defining our environment 

### VizDoom

In [5]:
#Define our vizdoom environment class
class VizDoomGym(Env):
    #Initialize our environment
    def __init__(self, render=False):
        #inherit from env base class
        super().__init__()
        self.game = DoomGame()
        #This allows us to load up our configurations which defines our maps,rewards,buttons etc...
        self.game.load_config('github/ViZDoom/scenarios/basic.cfg')
        
        
        #Determine if to render game window
        if render == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
        
        #start up the game
        self.game.init()
        
        #create our observation space.
        #We want the same of the observation space to match the game frame exactly- 
        #This is what is used to establish the parameters for the underlying models.
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8)
        #define our action space
        self.action_space = Discrete(3)
    #take step in environment
    def step(self, action):
        #Define the action to take
        actions = np.identity(3, dtype=np.uint8)
        #this actions will be a matrix defining if the agent go left,
        #right or shoot and also our frame skip parameter
        reward = self.game.make_action(actions[action],4)        
        
        #return numpy zeroes array if nothing is returned
        if self.game.get_state():
            state = self.game.get_state().screen_buffer
            #gray scaling the captured image
            state = self.grayscale(state)
            ammo = self.game.get_state().game_variables[0]
            info = ammo
        else:
            state = np.zeros(self.observation_space.shape)
            info = 0      
        
        
        
        info = {"info":info}
        done = self.game.is_episode_finished()
        
        return state, reward, done, info
    #render game 
    def render():
        pass    
    def reset(self):
        self.game.new_episode()        
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)
    
    #grayscale and resize the image
    def grayscale(self, observation):
        #take the observation, grab the color channel and move it to the end
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state
    #close the game
    def close(self):
        #self.game.close()
        pass

### Verify our environment

In [6]:
#import environment checker
from stable_baselines3.common import env_checker

In [12]:
env = VizDoomGym(render=True)

In [13]:
env_checker.check_env(env)

In [15]:
env.close()

In [16]:
#create our environment without rendering it
env = VizDoomGym()

### Defining custom DQN Network

In [30]:
class CustomCNN(BaseFeaturesExtractor):
    
    
    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 256):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        # We assume CxHxW images (channels first)
        # Re-ordering will be done by pre-preprocessing or wrapper
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(
                th.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))
    

policy_kwargs = dict(
    features_extractor_class=CustomCNN,    
)

### Setting up Callback

In [31]:
class TrainingAndLoggingCallback(BaseCallback):
    
   
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainingAndLoggingCallback, self). __init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
            
            
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
            
        return True

In [32]:
CHECKPOINT_DIR = './train/train_basic3'
LOG_DIR = './train/log_basic3'

In [33]:
callback = TrainingAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

### Create and Train model

In [34]:
 model = DQN("CnnPolicy", env, buffer_size = 320000,batch_size = 64, policy_kwargs=policy_kwargs,
             verbose=1,optimize_memory_usage = True, learning_rate=0.001)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [35]:
model.learn(total_timesteps=100000,callback=callback)

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 56.8     |
|    ep_rew_mean      | -252     |
|    exploration_rate | 0.978    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 656      |
|    time_elapsed     | 0        |
|    total_timesteps  | 227      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 47.8     |
|    ep_rew_mean      | -179     |
|    exploration_rate | 0.964    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 748      |
|    time_elapsed     | 0        |
|    total_timesteps  | 382      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 40.7     |
|    ep_rew_mean      | -128     |
|    exploration_rate | 0.954    |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 40.5     |
|    ep_rew_mean      | -134     |
|    exploration_rate | 0.646    |
| time/               |          |
|    episodes         | 92       |
|    fps              | 783      |
|    time_elapsed     | 4        |
|    total_timesteps  | 3727     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 40.2     |
|    ep_rew_mean      | -132     |
|    exploration_rate | 0.633    |
| time/               |          |
|    episodes         | 96       |
|    fps              | 787      |
|    time_elapsed     | 4        |
|    total_timesteps  | 3860     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 40.3     |
|    ep_rew_mean      | -133     |
|    exploration_rate | 0.618    |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 33.7     |
|    ep_rew_mean      | -90.9    |
|    exploration_rate | 0.374    |
| time/               |          |
|    episodes         | 180      |
|    fps              | 840      |
|    time_elapsed     | 7        |
|    total_timesteps  | 6586     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 32.4     |
|    ep_rew_mean      | -83.7    |
|    exploration_rate | 0.373    |
| time/               |          |
|    episodes         | 184      |
|    fps              | 839      |
|    time_elapsed     | 7        |
|    total_timesteps  | 6600     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 32.4     |
|    ep_rew_mean      | -82.2    |
|    exploration_rate | 0.352    |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 35.4     |
|    ep_rew_mean      | -99.8    |
|    exploration_rate | 0.0696   |
| time/               |          |
|    episodes         | 268      |
|    fps              | 850      |
|    time_elapsed     | 11       |
|    total_timesteps  | 9794     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 34.4     |
|    ep_rew_mean      | -92.8    |
|    exploration_rate | 0.0631   |
| time/               |          |
|    episodes         | 272      |
|    fps              | 839      |
|    time_elapsed     | 11       |
|    total_timesteps  | 9862     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 34.4     |
|    ep_rew_mean      | -92.6    |
|    exploration_rate | 0.0594   |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 29.6     |
|    ep_rew_mean      | -68.2    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 356      |
|    fps              | 840      |
|    time_elapsed     | 14       |
|    total_timesteps  | 12319    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 29.8     |
|    ep_rew_mean      | -69.4    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 360      |
|    fps              | 842      |
|    time_elapsed     | 14       |
|    total_timesteps  | 12495    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 30.9     |
|    ep_rew_mean      | -76.5    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

...

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 19.7     |
|    ep_rew_mean      | 4.49     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 3072     |
|    fps              | 113      |
|    time_elapsed     | 755      |
|    total_timesteps  | 85742    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 6.14     |
|    n_updates        | 8935     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 19.9     |
|    ep_rew_mean      | 3.55     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 3076     |
|    fps              | 113      |
|    time_elapsed     | 756      |
|    total_timesteps  | 85786    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 2.05     |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 20.3     |
|    ep_rew_mean      | 1.91     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 3136     |
|    fps              | 111      |
|    time_elapsed     | 778      |
|    total_timesteps  | 86887    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.879    |
|    n_updates        | 9221     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 20.3     |
|    ep_rew_mean      | 1.99     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 3140     |
|    fps              | 111      |
|    time_elapsed     | 778      |
|    total_timesteps  | 86898    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 2.61     |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 20.3     |
|    ep_rew_mean      | 0.01     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 3200     |
|    fps              | 109      |
|    time_elapsed     | 804      |
|    total_timesteps  | 88143    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.91     |
|    n_updates        | 9535     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 20.3     |
|    ep_rew_mean      | 0.05     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 3204     |
|    fps              | 109      |
|    time_elapsed     | 804      |
|    total_timesteps  | 88157    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 2.18     |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 16.7     |
|    ep_rew_mean      | 18.4     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 3264     |
|    fps              | 108      |
|    time_elapsed     | 822      |
|    total_timesteps  | 88969    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 1.98     |
|    n_updates        | 9742     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 15.9     |
|    ep_rew_mean      | 21.5     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 3268     |
|    fps              | 108      |
|    time_elapsed     | 824      |
|    total_timesteps  | 89055    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 1.91     |
|    n_updates      

KeyboardInterrupt: 

## Optuna for hyperparameter tuning

### Dependencies for optuna

In [37]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler

### Optuna variables

In [39]:
N_TRIALS = 100
N_STARTUP_TRIALS = 10
N_EVALUATIONS = 2
N_TIMESTEPS = int(2e4)
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_EPISODES = 3
DEFAULT_HYPERPARAMS = {
    "policy": "CnnPolicy",
    "env": env,
}

### Optuna Sampler

In [40]:
def sample_DQN_params(trial: optuna.Trial) -> Dict[str, Any]:
    
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    buffer_size = trial.suggest_categorical("buffer_size ",[int(1e3),int(1e4),int(1e5)])    
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64,128])    
    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])
    seed  = trial.suggest_categorical("seed ", [1, 2,3,4,5])
    optimize_memory_usage = True

    # Display true values
    trial.set_user_attr("gamma_", gamma)    

    net_arch = [
        16,32 if net_arch == "tiny" else 32,32,32
    ]

    activation_fn = { "relu": nn.ReLU, "tanh": nn.Tanh,}[activation_fn]

    return {        
        "gamma": gamma,
        "buffer_size": buffer_size,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "max_grad_norm": max_grad_norm,
        "optimize_memory_usage": optimize_memory_usage,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,            
        },
    }

### Callback for evaluating trial

In [41]:
class TrialEvalCallback(EvalCallback):
    
    def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            super()._on_step()
            self.eval_idx += 1
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True

### Model Tuning

In [42]:
def objective(trial: optuna.Trial) -> float:

    kwargs = DEFAULT_HYPERPARAMS.copy()
    # Sample hyperparameters
    kwargs.update(sample_DQN_params(trial))
    # Create our model
    model = DQN(**kwargs)
    # Wrapping env used for evaluation
    eval_env = Monitor(env)
    eval_env = DummyVecEnv([lambda: eval_env])
    eval_env = VecTransposeImage(eval_env)    
    # Create the callback that will periodically evaluate
    # and report the performance
    eval_callback = TrialEvalCallback(
        eval_env, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ, deterministic=True
    )

    nan_encountered = False
    try:
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN
        print(e)
        nan_encountered = True
    finally:
        # Free memory
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward


if __name__ == "__main__":
    # Set pytorch num threads to 1 for faster training
    torch.set_num_threads(1)

    sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
    # Do not prune before 1/3 of the max budget is used
    pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3)

    study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
    try:
        study.optimize(objective, n_trials=N_TRIALS, timeout=600)
    except KeyboardInterrupt:
        pass

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print("    {}: {}".format(key, value))

[I 2022-04-19 22:48:33,765] A new study created in memory with name: no-name-db4344f9-1bdf-4fb4-a5ba-c905241cd224
[I 2022-04-19 22:48:56,484] Trial 0 finished with value: -300.0 and parameters: {'gamma': 0.0009650797921447411, 'buffer_size ': 1000, 'lr': 0.04056457586217798, 'max_grad_norm': 4.269871692268495, 'batch_size': 64, 'net_arch': 'tiny', 'activation_fn': 'relu', 'seed ': 2}. Best is trial 0 with value: -300.0.
[I 2022-04-19 22:49:20,502] Trial 1 finished with value: -300.0 and parameters: {'gamma': 0.0020954332430213047, 'buffer_size ': 100000, 'lr': 7.401715439223582e-05, 'max_grad_norm': 2.7585881320921772, 'batch_size': 32, 'net_arch': 'tiny', 'activation_fn': 'relu', 'seed ': 4}. Best is trial 0 with value: -300.0.
[I 2022-04-19 22:49:44,216] Trial 2 finished with value: -300.0 and parameters: {'gamma': 0.0755636059450642, 'buffer_size ': 100000, 'lr': 0.10040713616491531, 'max_grad_norm': 0.8752237129600708, 'batch_size': 64, 'net_arch': 'tiny', 'activation_fn': 'relu', 

[I 2022-04-19 22:57:54,520] Trial 25 finished with value: -73.33333333333333 and parameters: {'gamma': 0.005630025133523602, 'buffer_size ': 10000, 'lr': 5.899082903004146e-05, 'max_grad_norm': 1.1354562766442862, 'batch_size': 128, 'net_arch': 'small', 'activation_fn': 'tanh', 'seed ': 1}. Best is trial 6 with value: 95.0.
[I 2022-04-19 22:58:17,556] Trial 26 finished with value: -300.0 and parameters: {'gamma': 0.0004705877190887067, 'buffer_size ': 100000, 'lr': 2.5394704902101727e-05, 'max_grad_norm': 2.9542213610448136, 'batch_size': 64, 'net_arch': 'small', 'activation_fn': 'relu', 'seed ': 4}. Best is trial 6 with value: 95.0.
[I 2022-04-19 22:58:35,468] Trial 27 finished with value: 82.33333333333333 and parameters: {'gamma': 0.002412684156061744, 'buffer_size ': 1000, 'lr': 0.0011893986200722858, 'max_grad_norm': 0.7284822446429858, 'batch_size': 32, 'net_arch': 'small', 'activation_fn': 'tanh', 'seed ': 1}. Best is trial 6 with value: 95.0.


Number of finished trials:  28
Best trial:
  Value:  95.0
  Params: 
    gamma: 0.0021813151695808546
    buffer_size : 100000
    lr: 0.00015581771449402266
    max_grad_norm: 2.8095652180149444
    batch_size: 64
    net_arch: small
    activation_fn: tanh
    seed : 1
  User attrs:
    gamma_: 0.9978186848304191


## Create and train our tuned model

Having completed our hyperparameter search, we plug in the hyperparameters generated from the search into our model along with the network archtecture values.

We supply the searched network architecture, orth_init and activation function using policy kwargs

In [48]:
policy_kwargs = dict(    
    activation_fn = th.nn.Tanh,
    net_arch = [32,32,64],
)

Create our model using searched hyperparameters and policy kwargs

In [49]:
model = DQN("CnnPolicy", env, tensorboard_log=LOG_DIR, verbose=1,
           gamma = 0.9978186848304191, max_grad_norm = 2.8095652180149444,
           buffer_size = 100000, learning_rate = 0.00015581771449402266,
           batch_size = 64,seed = 1,optimize_memory_usage = True, policy_kwargs=policy_kwargs)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


Train our agent using a specified number of timesteps

In [50]:
model.learn(total_timesteps=100000,callback=callback)

Logging to ./train/log_basic3\DQN_9
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.25     |
|    ep_rew_mean      | 84.8     |
|    exploration_rate | 0.998    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 293      |
|    time_elapsed     | 0        |
|    total_timesteps  | 17       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 13.9     |
|    ep_rew_mean      | 24.1     |
|    exploration_rate | 0.989    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 772      |
|    time_elapsed     | 0        |
|    total_timesteps  | 111      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 25.5     |
|    ep_rew_mean      | -46.2    |
|    exploration_rate | 0.971    |
| time/            

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 27.2     |
|    ep_rew_mean      | -48.1    |
|    exploration_rate | 0.762    |
| time/               |          |
|    episodes         | 92       |
|    fps              | 1027     |
|    time_elapsed     | 2        |
|    total_timesteps  | 2506     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 27.3     |
|    ep_rew_mean      | -48.4    |
|    exploration_rate | 0.751    |
| time/               |          |
|    episodes         | 96       |
|    fps              | 1030     |
|    time_elapsed     | 2        |
|    total_timesteps  | 2619     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 27.7     |
|    ep_rew_mean      | -52      |
|    exploration_rate | 0.736    |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 33.8     |
|    ep_rew_mean      | -89      |
|    exploration_rate | 0.48     |
| time/               |          |
|    episodes         | 180      |
|    fps              | 1033     |
|    time_elapsed     | 5        |
|    total_timesteps  | 5471     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 32.5     |
|    ep_rew_mean      | -80.8    |
|    exploration_rate | 0.471    |
| time/               |          |
|    episodes         | 184      |
|    fps              | 1035     |
|    time_elapsed     | 5        |
|    total_timesteps  | 5572     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 33.7     |
|    ep_rew_mean      | -87.8    |
|    exploration_rate | 0.453    |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 32.6     |
|    ep_rew_mean      | -81.5    |
|    exploration_rate | 0.205    |
| time/               |          |
|    episodes         | 268      |
|    fps              | 973      |
|    time_elapsed     | 8        |
|    total_timesteps  | 8369     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 32       |
|    ep_rew_mean      | -77.5    |
|    exploration_rate | 0.197    |
| time/               |          |
|    episodes         | 272      |
|    fps              | 974      |
|    time_elapsed     | 8        |
|    total_timesteps  | 8453     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 30.3     |
|    ep_rew_mean      | -67.3    |
|    exploration_rate | 0.196    |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 20.2     |
|    ep_rew_mean      | -7.06    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 356      |
|    fps              | 974      |
|    time_elapsed     | 10       |
|    total_timesteps  | 10162    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 20.8     |
|    ep_rew_mean      | -10.9    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 360      |
|    fps              | 971      |
|    time_elapsed     | 10       |
|    total_timesteps  | 10271    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.2     |
|    ep_rew_mean      | -12.8    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

...

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 62.2     |
|    ep_rew_mean      | -241     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1988     |
|    fps              | 153      |
|    time_elapsed     | 467      |
|    total_timesteps  | 71994    |
| train/              |          |
|    learning_rate    | 0.000156 |
|    loss             | 3.95     |
|    n_updates        | 5498     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 62.4     |
|    ep_rew_mean      | -243     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1992     |
|    fps              | 152      |
|    time_elapsed     | 473      |
|    total_timesteps  | 72294    |
| train/              |          |
|    learning_rate    | 0.000156 |
|    loss             | 0.567    |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 63.9     |
|    ep_rew_mean      | -246     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 2052     |
|    fps              | 137      |
|    time_elapsed     | 557      |
|    total_timesteps  | 76684    |
| train/              |          |
|    learning_rate    | 0.000156 |
|    loss             | 0.462    |
|    n_updates        | 6670     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 66.1     |
|    ep_rew_mean      | -257     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 2056     |
|    fps              | 136      |
|    time_elapsed     | 562      |
|    total_timesteps  | 76984    |
| train/              |          |
|    learning_rate    | 0.000156 |
|    loss             | 0.143    |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 62.3     |
|    ep_rew_mean      | -236     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 2116     |
|    fps              | 128      |
|    time_elapsed     | 624      |
|    total_timesteps  | 80252    |
| train/              |          |
|    learning_rate    | 0.000156 |
|    loss             | 2.25     |
|    n_updates        | 7562     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 59.4     |
|    ep_rew_mean      | -220     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 2120     |
|    fps              | 128      |
|    time_elapsed     | 625      |
|    total_timesteps  | 80260    |
| train/              |          |
|    learning_rate    | 0.000156 |
|    loss             | 3.61     |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 49       |
|    ep_rew_mean      | -166     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 2180     |
|    fps              | 120      |
|    time_elapsed     | 692      |
|    total_timesteps  | 83546    |
| train/              |          |
|    learning_rate    | 0.000156 |
|    loss             | 4.39     |
|    n_updates        | 8386     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 49       |
|    ep_rew_mean      | -166     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 2184     |
|    fps              | 119      |
|    time_elapsed     | 698      |
|    total_timesteps  | 83846    |
| train/              |          |
|    learning_rate    | 0.000156 |
|    loss             | 0.76     |
|    n_updates      

KeyboardInterrupt: 

### Test and evaluate our model

We will evaluate our model in the video game environment to determine its performance

We load our best trained model to use for evaluation

In [ ]:
model = PPO.load('./train/train_basic2/best_model_80000')

Create and render our environment to see the agents performance

Create and render our environment to see the agents performance

In [ ]:
env = VizDoomGym(render=True)

In [ ]:
#loop through each game
for episode in range(10):
    obs = env.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        time.sleep(0.20)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(total_reward, episode))
    time.sleep(2)